### Get burnt geopoints

In [51]:
import json
import csv
import shapely

# # Example of the CSV file content (assuming it's already read as lines)
# csv_file_path = '/Users/sonavagarwal/Documents/GitHub/hackmit-24/gather-training-data/BurnedAreaPatch2.csv'

# # List to hold the geopoints
# burnt_geopoints = []

# # Open and parse the CSV file
# with open(csv_file_path, newline='') as csvfile:
#     reader = csv.reader(csvfile)
    
#     # Skip the header
#     next(reader)
    
#     for row in reader:
#         if (len(row) == 4):
#             # The GeoJSON-like data is in the fourth column (row[3])
#             geojson_data = json.loads(row[3])
            
#             # turn geojson into shape 
#             shape = shapely.geometry.shape(geojson_data)

#             # return the centroid of the shape
#             centroid = shape.centroid

#             # Append the centroid to the list
#             lat = centroid.y
#             lon = centroid.x
#             burnt_geopoints.append((lat, lon))

# # Now burnt_geopoints contains all the points from the polygons
# # Example: print the first few geopoints

# print("len(burnt_geopoints): ", len(burnt_geopoints))
# print(burnt_geopoints[:5])


# import random

# def generate_random_coordinates_in_paradise():
#     # Coordinates of the upper left and bottom right points
#     upper_left_lat = 39.775281210424154
#     upper_left_lon = -121.62160339203449
#     bottom_right_lat = 39.75133027591371
#     bottom_right_lon = -121.5813038320607
    
#     # Generate random latitude and longitude within the bounds
#     random_lat = random.uniform(bottom_right_lat, upper_left_lat)
#     random_lon = random.uniform(upper_left_lon, bottom_right_lon)
    
#     return random_lat, random_lon


# import json

# def calculate_dimensions(rings):
#     """
#     Calculate the width and height of the parcel based on the bounding box of the coordinates.
#     """
#     x_coords = [point[0] for ring in rings for point in ring]
#     y_coords = [point[1] for ring in rings for point in ring]

#     width = max(x_coords) - min(x_coords)
#     height = max(y_coords) - min(y_coords)

#     return width, height

# def process_paradise_houses(file_path):
#     """
#     Process the JSON file and calculate centroid, width, and height for each parcel.
#     """
#     with open(file_path, 'r') as f:
#         data = json.load(f)

#     parcels = []

#     for feature in data['features']:
#         # Get the rings (polygon coordinates) and centroid
#         rings = feature['geometry']['rings']
#         centroid = feature['centroid']

#         # Calculate width and height
#         width, height = calculate_dimensions(rings)

#         # Append the result as a tuple (centroid, width, height)
#         parcels.append((centroid, width, height))

#     return parcels

# # Specify the path to the JSON file
# file_path = '/Users/sonavagarwal/Documents/GitHub/hackmit-24/gather-training-data/paradise_houses.json'

# # Process the file and get the results
# burnt_parcels = process_paradise_houses(file_path)

# # Output the results
# for parcel in parcels:
#     print(f"Centroid: {parcel[0]}, Width: {parcel[1]:.2f}, Height: {parcel[2]:.2f}")

import json
from pyproj import Transformer

def calculate_dimensions(rings):
    """
    Calculate the width and height of the parcel based on the bounding box of the coordinates.
    """
    x_coords = [point[0] for ring in rings for point in ring]
    y_coords = [point[1] for ring in rings for point in ring]

    width = max(x_coords) - min(x_coords)
    height = max(y_coords) - min(y_coords)

    return width, height

def convert_to_latlong(x, y, transformer):
    """
    Convert coordinates from California State Plane (EPSG:2226) to WGS84 (EPSG:4326).
    """
    lon, lat = transformer.transform(x, y)
    return lat, lon

def process_paradise_houses(file_path):
    """
    Process the JSON file and calculate centroid (converted to lat/long), width, and height for each parcel.
    """
    # Initialize the transformer from EPSG:2226 (State Plane) to EPSG:4326 (WGS84)
    transformer = Transformer.from_crs(2226, 4326, always_xy=True)

    with open(file_path, 'r') as f:
        data = json.load(f)

    parcels = []

    for feature in data['features']:
        # Get the rings (polygon coordinates) and centroid
        rings = feature['geometry']['rings']
        centroid = feature['centroid']

        # Convert the centroid to lat/long
        lat, lon = convert_to_latlong(centroid['x'], centroid['y'], transformer)

        # Calculate width and height
        width, height = calculate_dimensions(rings)

        # Append the result as a tuple (lat/long centroid, width, height)
        parcels.append(((lat, lon), width, height))

    return parcels

# Specify the path to the JSON file
file_path = '/Users/sonavagarwal/Documents/GitHub/hackmit-24/gather-training-data/paradise_houses.json'

# Process the file and get the results
parcels = process_paradise_houses(file_path)

# Output the results
for parcel in parcels:
    print(f"Centroid (Lat, Long): {parcel[0]}, Width: {parcel[1]:.2f}, Height: {parcel[2]:.2f}")



Centroid (Lat, Long): (39.75386924740615, -121.61133802949706), Width: 133.76, Height: 151.87
Centroid (Lat, Long): (39.75978522987245, -121.60489119506278), Width: 210.80, Height: 142.31
Centroid (Lat, Long): (39.76183995669749, -121.58736007807022), Width: 126.39, Height: 280.66
Centroid (Lat, Long): (39.7599901500105, -121.59348656458336), Width: 100.33, Height: 83.28
Centroid (Lat, Long): (39.750783670238874, -121.58782733971327), Width: 278.50, Height: 265.55
Centroid (Lat, Long): (39.77724180922268, -121.6085597956986), Width: 155.00, Height: 224.59
Centroid (Lat, Long): (39.76193091891568, -121.57881998823544), Width: 467.78, Height: 302.68
Centroid (Lat, Long): (39.763570765514, -121.62464759478952), Width: 229.95, Height: 105.24
Centroid (Lat, Long): (39.75442809344649, -121.58542099088459), Width: 84.51, Height: 209.90
Centroid (Lat, Long): (39.769645400826064, -121.62250888493793), Width: 153.13, Height: 385.32
Centroid (Lat, Long): (39.772634804175375, -121.62562346286906),

### Get not burnt geopoints

In [15]:
import random

def generate_random_coordinates_in_tracy():
    # Coordinates of the upper left and bottom right points
    upper_left_lat = 37.753769564438734
    upper_left_lon = -121.46080223281191
    bottom_right_lat = 37.72498664651281
    bottom_right_lon = -121.41662178760858
    
    # Generate random latitude and longitude within the bounds
    random_lat = random.uniform(bottom_right_lat, upper_left_lat)
    random_lon = random.uniform(upper_left_lon, bottom_right_lon)
    
    return random_lat, random_lon


In [16]:

num_points = 1000

burnt_coordinates = [generate_random_coordinates_in_paradise() for _ in range(num_points)]
safe_coordinates = [generate_random_coordinates_in_tracy() for _ in range(num_points)]

print("len(burnt_coordinates):", len(burnt_coordinates))
print("len(safe_coordinates):", len(safe_coordinates))

len(burnt_coordinates): 1000
len(safe_coordinates): 1000


In [17]:
for burnt in burnt_coordinates[:5]:
    print(burnt)
for safe in safe_coordinates[:5]:
    print(safe)

(39.76573798852989, -121.60600354688104)
(39.75392407921986, -121.61363635060921)
(39.7724469703868, -121.61357203597909)
(39.7736674623969, -121.60156028427966)
(39.75345613579683, -121.59267378685983)
(37.73600717682408, -121.45700594710426)
(37.74953723117653, -121.42737111471912)
(37.75148175244209, -121.42885046522562)
(37.75057136574473, -121.42699085041951)
(37.72869866451546, -121.44347103276219)


### Get lists of coordinates of houses in certain areas

In [24]:
import osmnx as ox

# only use data from before 2018
ox.settings.overpass_settings = '[out:json][timeout:180][date:"2023-01-01T00:00:00Z"]'

# Function to get houses near a specific coordinate within a given distance
def get_houses_near_coordinate(lat, lon, dist=500):
    """
    Get house footprints near a specific coordinate within a given radius.

    Parameters:
        lat (float): Latitude of the center point.
        lon (float): Longitude of the center point.
        dist (int): Radius in meters to search around the point. Defaults to 500 meters.

    Returns:
        GeoDataFrame: A GeoDataFrame containing house footprints near the coordinate.
    """
    # Create a point from the provided coordinates
    point = (lat, lon)
    
    try:
        # Use the correct OSMnx function to get geometries around a point within the specified distance
        gdf = ox.features_from_point(point, tags={'building': True}, dist=dist)
    
        return gdf
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to extract house centroids and their approximate dimensions (width and height)
def extract_house_centroids_and_dimensions(gdf):
    """
    Extract centroids, widths, and heights from a GeoDataFrame of house footprints.

    Parameters:
        gdf (GeoDataFrame): A GeoDataFrame containing house footprints.

    Returns:
        DataFrame: A DataFrame containing the centroids, widths, and heights of the houses.
    """

    # Compute centroids before changing the CRS (to keep in lat/lon)
    gdf['centroid'] = gdf.centroid
    
    # Extract lat/lon from the centroids and ensure correct order (lat, lon)
    gdf['centroid_lat'] = gdf['centroid'].apply(lambda point: point.y)
    gdf['centroid_lon'] = gdf['centroid'].apply(lambda point: point.x)
    
    # Project the GeoDataFrame to UTM (or a local CRS that uses meters for accurate width/height)
    gdf_meters = gdf.to_crs(epsg=3857)  # Web Mercator (meters)
    
    # Compute approximate width and height of each house (bounding box dimensions) in meters
    gdf_meters['width'] = gdf_meters.bounds['maxx'] - gdf_meters.bounds['minx']
    gdf_meters['height'] = gdf_meters.bounds['maxy'] - gdf_meters.bounds['miny']
    
    # Create a DataFrame with centroids (lat/lon) and dimensions in meters
    house_info = gdf_meters[['width', 'height']].copy()
    house_info['centroid_lat'] = gdf['centroid_lat']
    house_info['centroid_lon'] = gdf['centroid_lon']
    
    return house_info

# Example usage:
# lat, lon = 37.36410775117878, -122.0953579301575
lat, lon = 20.873532094021396, -156.6761566297102
houses_gdf = get_houses_near_coordinate(lat, lon, dist=20)
house_info = extract_house_centroids_and_dimensions(houses_gdf)

# Print the results
print(house_info)


                            width     height  centroid_lat  centroid_lon
element_type osmid                                                      
way          877462453  24.434628  23.267737     20.873254   -156.676007
             877462454  16.786979  18.788141     20.873323   -156.675892
             877561083  14.850020  13.426937     20.873644   -156.675943
             877561084  14.961340  16.059914     20.873743   -156.676036
             877561085  15.228506  16.226691     20.873580   -156.676147
             877561086  14.638513  15.654821     20.873544   -156.676286


/var/folders/1g/yznzy8852x1cbyhn96cd3r1r0000gn/T/ipykernel_35999/3809493269.py:44: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.centroid


In [21]:
import pandas as pd

# empty dataframe to store all the burnt houses
all_burnt_houses = []

# for burnt in burnt_coordinates:
#     if len(all_burnt_houses) >= 10:  break
#     print(burnt)
#     lat, lon = burnt
#     houses_gdf = get_houses_near_coordinate(lat, lon, dist=200)
#     if houses_gdf is None:
#         continue
#     house_info = extract_house_centroids_and_dimensions(houses_gdf)
#     for index, row in house_info.iterrows():
#         all_burnt_houses.append((row['centroid_lat'], row['centroid_lon'], row['width'], row['height']))
#         print(row['centroid_lat'], row['centroid_lon'], row['width'], row['height'])

# empty dataframe to store all the safe houses
all_safe_houses = []
for safe in safe_coordinates:
    if len(all_safe_houses) >= 10:  break
    lat, lon = safe
    houses_gdf = get_houses_near_coordinate(lat, lon, dist=20)
    if houses_gdf is None:
        continue
    house_info = extract_house_centroids_and_dimensions(houses_gdf)
    for index, row in house_info.iterrows():
        all_safe_houses.append((row['centroid_lat'], row['centroid_lon'], row['width'], row['height']))


An error occurred: No data elements in server response. Check log and query location/tags.
An error occurred: No data elements in server response. Check log and query location/tags.
An error occurred: No data elements in server response. Check log and query location/tags.


KeyboardInterrupt: 

In [9]:
for burnt in all_burnt_houses[:5]:
    print(burnt)

print("\n\n")
for safe in all_safe_houses[:5]:
    print(safe)

### get images from list of coordinates

In [52]:
import pyproj

def calculate_bounding_box(lat, lon, width, height):
    """
    Calculate the bounding box in Web Mercator coordinates (EPSG:3857) 
    given a center latitude and longitude, along with the desired width and height in meters.
    
    Parameters:
    lat (float): Latitude of the center point.
    lon (float): Longitude of the center point.
    width (float): Width of the bounding box in meters.
    height (float): Height of the bounding box in meters.

    Returns:
    tuple: Bounding box as (xmin, ymin, xmax, ymax) in meters.
    """
    # Define the Web Mercator (EPSG:3857) projection
    web_mercator = pyproj.Proj(init='epsg:3857')
    
    # Convert the center point to Web Mercator (meters)
    x_center, y_center = web_mercator(lon, lat)
    
    # Calculate the half width and half height to determine the bounding box
    half_width = width / 2
    half_height = height / 2
    
    # Calculate the bounding box in meters
    x_min = x_center - half_width
    x_max = x_center + half_width
    y_min = y_center - half_height
    y_max = y_center + half_height
    
    return (x_min, y_min, x_max, y_max)

def calc_bounding_box_from_wm(centroid, width, height):
    """
    Calculate the bounding box in Web Mercator coordinates (EPSG:3857) 
    given a center latitude and longitude, along with the desired width and height in meters.
    
    Parameters:
    centroid (tuple): Latitude and longitude of the centroid.
    width (float): Width of the bounding box in meters.
    height (float): Height of the bounding box in meters.

    Returns:
    tuple: Bounding box as (xmin, ymin, xmax, ymax) in meters.
    """
    
    x_center, y_center = centroid["x"], centroid["y"]

    x_min = x_center - width / 2
    x_max = x_center + width / 2
    y_min = y_center - height / 2
    y_max = y_center + height / 2

    return (x_min, y_min, x_max, y_max)


import urllib.parse

def construct_naip_url_encoded(bbox):
    """
    Construct the NAIP image export URL using the bounding box coordinates, with URL encoding.

    Parameters:
    bbox (tuple): Bounding box as (xmin, ymin, xmax, ymax) in meters.

    Returns:
    str: The formatted and URL-encoded NAIP exportImage URL with the bounding box.
    """
    base_url = "https://map.dfg.ca.gov/arcgis/rest/services/Base_Remote_Sensing/NAIP_2016/ImageServer/exportImage"
    bbox_str = f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}"
    
    # URL encode the bounding box string
    bbox_encoded = urllib.parse.quote(bbox_str)
    
    # Construct the full URL with the bounding box
    url = f"{base_url}?bbox={bbox_encoded}&bboxSR=&size=&imageSR=&time=&format=jpgpng&pixelType=U8&noData=&noDataInterpretation=esriNoDataMatchAny&interpolation=+RSP_BilinearInterpolation&compression=&compressionQuality=&bandIds=&mosaicRule=&renderingRule=&f=image"
    
    return url

lat, lon = 38.8073838, -120.1060273
# lat, lon = 39.76823175240576, -121.61587209906226
# lat, lon = 39.76249836942643, -121.60853199062909

# Get the bounding box for a house with a width of 10 meters and height of 10 meters
bbox = calculate_bounding_box(lat, lon, 50, 500)
print(bbox)

# Construct the NAIP image export URL with the bounding box
url = construct_naip_url_encoded(bbox)
print(url)



(-13370166.800239014, 4693868.390095745, -13370116.800239014, 4694368.390095745)
https://map.dfg.ca.gov/arcgis/rest/services/Base_Remote_Sensing/NAIP_2016/ImageServer/exportImage?bbox=-13370166.800239014%2C4693868.390095745%2C-13370116.800239014%2C4694368.390095745&bboxSR=&size=&imageSR=&time=&format=jpgpng&pixelType=U8&noData=&noDataInterpretation=esriNoDataMatchAny&interpolation=+RSP_BilinearInterpolation&compression=&compressionQuality=&bandIds=&mosaicRule=&renderingRule=&f=image


/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [53]:
burnt_parcel_info = []
for burnt_parcel in burnt_parcels[:5]:
    # Get the centroid and dimensions of the parcel
    centroid = burnt_parcel[0]
    width = burnt_parcel[1]
    height = burnt_parcel[2]
    print(centroid)
    
    bp_bbox = calculate_bounding_box(centroid["x"], centroid["y"], 300, 300)
    bp_url = construct_naip_url_encoded(bp_bbox)

    burnt_parcel_info.append((centroid, width, height, bp_url))

for bp in burnt_parcel_info[:5]:
    print(bp)

    


{'x': 6670943.3659071475, 'y': 2400812.705296808}
{'x': 6672746.590548394, 'y': 2402975.5043034367}
{'x': 6677671.824323099, 'y': 2403745.911710975}
{'x': 6675952.511720393, 'y': 2403064.3194081527}
{'x': 6677558.727288239, 'y': 2399717.9385400726}
({'x': 6670943.3659071475, 'y': 2400812.705296808}, 133.75891882181168, 151.86715033650398, 'https://map.dfg.ca.gov/arcgis/rest/services/Base_Remote_Sensing/NAIP_2016/ImageServer/exportImage?bbox=inf%2Cinf%2Cinf%2Cinf&bboxSR=&size=&imageSR=&time=&format=jpgpng&pixelType=U8&noData=&noDataInterpretation=esriNoDataMatchAny&interpolation=+RSP_BilinearInterpolation&compression=&compressionQuality=&bandIds=&mosaicRule=&renderingRule=&f=image')
({'x': 6672746.590548394, 'y': 2402975.5043034367}, 210.80469650030136, 142.30745816230774, 'https://map.dfg.ca.gov/arcgis/rest/services/Base_Remote_Sensing/NAIP_2016/ImageServer/exportImage?bbox=inf%2Cinf%2Cinf%2Cinf&bboxSR=&size=&imageSR=&time=&format=jpgpng&pixelType=U8&noData=&noDataInterpretation=esriNo

/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>

In [35]:
# pull property coordinates

from pyproj import Proj, transform
import urllib.parse

# Define projections
wgs84 = Proj(init='epsg:4326')  # WGS84 (lat/long)
state_plane = Proj(init='epsg:2226')  # NAD 1983 StatePlane California Zone V (feet)

# Coordinates in lat/long
top_left_lat, top_left_lon = 39.77719416735059, -121.62663831452272
bottom_right_lat, bottom_right_lon = 39.74530245745502, -121.57962767345121

# Convert to State Plane
top_left_x, top_left_y = transform(wgs84, state_plane, top_left_lon, top_left_lat)
bottom_right_x, bottom_right_y = transform(wgs84, state_plane, bottom_right_lon, bottom_right_lat)

# Base URL
base_url = "https://gisportal.buttecounty.net/arcgis/rest/services/Base_Layers/FeatureServer/1/query"

# Parameters for GET request
params = {
    "where": "1=1",
    "geometryType": "esriGeometryEnvelope",
    "geometry": {
        "xmin": top_left_x,
        "ymin": bottom_right_y,
        "xmax": bottom_right_x,
        "ymax": top_left_y,
        "spatialReference": {"wkid": 2226}
    },
    "inSR": 2226,
    "outSR": 4326,
    "outFields": "*",
    "f": "json"
}

# Convert geometry dictionary to a JSON-encoded string
params["geometry"] = urllib.parse.quote(str(params["geometry"]))

# Construct full URL
query_url = f"{base_url}?{urllib.parse.urlencode(params)}"


import numpy as np
import urllib.parse

# Function to create sub-areas
def generate_sub_area_urls(top_left_lat, top_left_lon, bottom_right_lat, bottom_right_lon, x_splits, y_splits):
    # Convert the coordinates to the State Plane system
    top_left_x, top_left_y = transform(wgs84, state_plane, top_left_lon, top_left_lat)
    bottom_right_x, bottom_right_y = transform(wgs84, state_plane, bottom_right_lon, bottom_right_lat)

    # Generate x and y step sizes
    x_steps = np.linspace(top_left_x, bottom_right_x, x_splits + 1)
    y_steps = np.linspace(bottom_right_y, top_left_y, y_splits + 1)

    # Generate the URLs for each sub-area
    urls = []
    for i in range(x_splits):
        for j in range(y_splits):
            # Define the bounding box for each sub-area
            xmin, xmax = x_steps[i], x_steps[i + 1]
            ymin, ymax = y_steps[j], y_steps[j + 1]

            # Create the geometry object
            geometry = {
                "xmin": xmin,
                "ymin": ymin,
                "xmax": xmax,
                "ymax": ymax,
                "spatialReference": {"wkid": 2226}
            }
            
            # Create the GET parameters
            params = {
                "where": "1=1",
                "geometryType": "esriGeometryEnvelope",
                "geometry": urllib.parse.quote(str(geometry)),
                "inSR": 2226,
                "outSR": 4326,
                "outFields": "*",
                "f": "json"
            }
            
            # Construct the full URL
            query_url = f"{base_url}?{urllib.parse.urlencode(params)}"
            urls.append(query_url)

    return urls

# Parameters for splitting
x_splits = 4  # Number of divisions along the x-axis
y_splits = 4  # Number of divisions along the y-axis

# Generate the URLs
sub_area_urls = generate_sub_area_urls(top_left_lat, top_left_lon, bottom_right_lat, bottom_right_lon, x_splits, y_splits)

sub_area_urls


/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/sonavagarwal/anaconda3/envs/maps/lib/python3.12/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/1g/yznzy8852x1cbyhn96cd3r1r0000gn/T/ipykernel_35999/2213791264.py:15: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/sta

['https://gisportal.buttecounty.net/arcgis/rest/services/Base_Layers/FeatureServer/1/query?where=1%3D1&geometryType=esriGeometryEnvelope&geometry=%257B%2527xmin%2527%253A%25206666606.632369177%252C%2520%2527ymin%2527%253A%25202397731.9247193914%252C%2520%2527xmax%2527%253A%25206669923.343918615%252C%2520%2527ymax%2527%253A%25202400621.704656698%252C%2520%2527spatialReference%2527%253A%2520%257B%2527wkid%2527%253A%25202226%257D%257D&inSR=2226&outSR=4326&outFields=%2A&f=json',
 'https://gisportal.buttecounty.net/arcgis/rest/services/Base_Layers/FeatureServer/1/query?where=1%3D1&geometryType=esriGeometryEnvelope&geometry=%257B%2527xmin%2527%253A%25206666606.632369177%252C%2520%2527ymin%2527%253A%25202400621.704656698%252C%2520%2527xmax%2527%253A%25206669923.343918615%252C%2520%2527ymax%2527%253A%25202403511.484594004%252C%2520%2527spatialReference%2527%253A%2520%257B%2527wkid%2527%253A%25202226%257D%257D&inSR=2226&outSR=4326&outFields=%2A&f=json',
 'https://gisportal.buttecounty.net/arcgi